In [1]:
# -*- coding: utf-8 -*-
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
from transformers import AutoTokenizer
# === 数据清洗与加载 ===
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1" 
def load_corpus(data_dir):
    """加载金庸小说语料库"""
    texts = []
    for fp in os.listdir(data_dir):
        if fp.endswith('.txt'):
            with open(os.path.join(data_dir, fp), 'r', encoding='gb18030') as f:
                text = re.sub(r'[\r\t\u3000]', '', f.read())  # 去除全角空格/制表符
                text = re.sub(r'\n{2,}', '\n', text)          # 合并多余空行
                texts.append(text)
    return '\n'.join(texts)[:500000]  # 限制语料规模以适应显存

# === 字符级处理（LSTM用）===
class CharDataset(Dataset):
    """字符级数据集"""
    def __init__(self, text, block_size=128):
        chars = sorted(list(set(text)))
        self.stoi = {ch:i+1 for i,ch in enumerate(chars)}  # 0为padding
        self.itos = {i:ch for ch,i in self.stoi.items()}
        self.ids = [self.stoi[ch] for ch in text]
        self.samples = [self.ids[i:i+block_size] 
                       for i in range(0, len(self.ids)-block_size, block_size//2)]
        
    def __len__(self): return len(self.samples)
    
    def __getitem__(self, idx):
        seq = self.samples[idx]
        x = torch.tensor(seq[:-1], dtype=torch.long)
        y = torch.tensor(seq[1:], dtype=torch.long)
        return x, y

# === 子词处理（Transformer用）===
class GPT2Dataset(Dataset):
    """改进后的分块逻辑"""
    def __init__(self, text, tokenizer, block_size=256):
        self.tokenizer = tokenizer
        # 增加分块前的截断处理（参考网页6）
        encoded = tokenizer.encode(text, max_length=block_size*1000, truncation=True)
        # 使用滑动窗口分块（参考网页8）
        self.samples = [encoded[i:i+block_size] 
                       for i in range(0, len(encoded)-block_size, block_size//2)]
    def __len__(self): return len(self.samples)
    
    def __getitem__(self, idx):
        seq = self.samples[idx]
        return torch.tensor(seq, dtype=torch.long)

# 初始化Tokenizer
#tokenizer = GPT2Tokenizer.from_pretrained("uer/gpt2-chinese-lyric")
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
text = "郭靖站起身来，只见远处黄沙漫天，隐约传来马蹄声响"
tokenizer = AutoTokenizer.from_pretrained("uer/gpt2-chinese-lyric")
# 强制截断到最大长度(512)
inputs = tokenizer(text, truncation=True, max_length=512, return_tensors="pt")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # 添加缺失的pad_token
# 数据加载示例
corpus = load_corpus('C:\文件\小说数据集')
lstm_ds = CharDataset(corpus)
gpt2_ds = GPT2Dataset(corpus, tokenizer)

In [2]:
class CharLSTM(torch.nn.Module):
    """基于字符的LSTM生成模型"""
    def __init__(self, vocab_size, emb_dim=256, hidden_size=512):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_size, num_layers=3, 
                                 dropout=0.2, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        return self.fc(out), hidden

# 训练配置
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lstm_model = CharLSTM(len(lstm_ds.stoi)+1).to(device)
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

# 数据加载器
lstm_dl = DataLoader(lstm_ds, batch_size=64, shuffle=True)

# 训练循环
for epoch in range(10):
    lstm_model.train()
    for x, y in lstm_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits, _ = lstm_model(x)
        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), 1.0)
        optimizer.step()
    print(f'Epoch {epoch} Loss: {loss.item():.3f}')

Epoch 0 Loss: 6.168
Epoch 1 Loss: 6.435
Epoch 2 Loss: 6.573
Epoch 3 Loss: 6.165
Epoch 4 Loss: 6.143
Epoch 5 Loss: 5.639
Epoch 6 Loss: 6.150
Epoch 7 Loss: 5.307
Epoch 8 Loss: 5.874
Epoch 9 Loss: 5.383


In [3]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
print(torch.__version__)        # 需≥2.5.1
print(torch.cuda.is_available()) # 需返回True
# 模型初始化
gpt2_model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-lyric")
gpt2_model.resize_token_embeddings(len(tokenizer))

# 训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=50,
    save_strategy="no"
)

# 自定义数据整理函数
def collate_fn(batch):
    return {'input_ids': torch.stack(batch), 
            'labels': torch.stack(batch)}

# 训练器
trainer = Trainer(
    model=gpt2_model,
    args=training_args,
    train_dataset=gpt2_ds,
    data_collator=collate_fn
)

# 开始微调
trainer.train()

2.7.0+cu128
True


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.130300
100,3.634800
150,3.477000


TrainOutput(global_step=186, training_loss=3.6838741507581485, metrics={'train_runtime': 42.8807, 'train_samples_per_second': 139.783, 'train_steps_per_second': 4.338, 'total_flos': 772901830656000.0, 'train_loss': 3.6838741507581485, 'epoch': 2.96})

In [4]:
def generate_lstm(model, start_text, length=500, temperature=0.8):
    """LSTM逐字符生成"""
    model.eval()
    chars = [lstm_ds.stoi.get(ch, 0) for ch in start_text]
    hidden = None
    with torch.no_grad():
        for _ in range(length):
            x = torch.tensor([chars[-1]], dtype=torch.long).unsqueeze(0).to(device)
            logits, hidden = model(x, hidden)
            prob = torch.softmax(logits[0,-1]/temperature, dim=-1)
            next_char = torch.multinomial(prob, 1).item()
            chars.append(next_char)
    return ''.join([lstm_ds.itos.get(c, '') for c in chars])

def generate_gpt2(model, start_text, length=500, temperature=0.9):
    """GPT-2生成"""
    input_ids = tokenizer.encode(start_text, return_tensors='pt').to(device)
    output = model.generate(
        input_ids,
        max_length=len(input_ids[0])+length,
        do_sample=True,
        temperature=temperature,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 生成示例对比
lstm_output = generate_lstm(lstm_model, start_text="郭靖站起身来")
gpt2_output = generate_gpt2(gpt2_model, start_text="郭靖站起身来")

print("=== LSTM生成结果 ===")
print(lstm_output[:500] + "...")
print("\n=== GPT-2生成结果 ===")
print(gpt2_output[:500] + "...")

=== LSTM生成结果 ===
郭靖站起身来狱，就是心刀，忽乾在是不北举开。头来又伸来出，一一酸的拉边。
李家洛想着言喜空头，实头已住，将手一前，我一台一骂，便是不意，过中疾惊，徒等撞钗的这口，不年上，帮方必了，听是是方起？”公过和道来中上，小心探过，待《长里着这语，性兵一住。那觉两人向他伤子，又不个说，见家道：“那时卫势这事在香他答，…用好截抱去，听讥对说在那旗说兵成门，会不在东，见我来辈兴功。”
陈家洛见一过一有两个齐来，要道：“俯里红来。不哥有的奔成。这人不么，不得好。”他哥》棋红，面目对看。
又是在健世在这出了。周菲仲指想上不生，在道：“我你我教便意。”他说：“总是放了。”天亲万叫：“咱要要，是我要想，”这人有不机之了，他把他来在不么，挺摇寡花，张这日大，要家大南？赵她用台火里一大，说得说拉，有是对四了打过，不位之声，不么，就若冲，然想到说来，大人在不借多上。”周沅洛道：“只想来就要回有，只然你来心敛，怎下一亲祭的，你兄么后拚儿见，有是率交蛋人的语。”徐绮桐道：“我果不能再去。那敢是逼叫便包度外，在这么你的我打层。”周菲青桐道：“我镖。”总卓同一起来。
陈家洛道：“各人要口，大然不人。他让连说“三么，我凉不有...

=== GPT-2生成结果 ===
郭 靖 站 起 身 来 ， ， 大 骆 驼 ， 骆 驼 越 来 越 近 。 他 的 手 脚 并 非 短 小 ， 但 轻 便 易 举 ， 一 旦 发 觉 自 己 有 一 腿 ， 越 是 怕 难 ， 就 别 再 给 他 手 脚 伤 了 。 骆 驼 手 的 眼 珠 ， 不 免 被 他 夺 了 半 点 ， 那 又 足 够 了 。 ， 白 龙 马 伸 出 马 尾 ， 说 道 ： 宝 贝 ， 我 们 一 定 尽 快 去 吧 。 骆 驼 上 树 又 下 河 ， 马 头 飞 起 ， 风 急 雨 急 ， 他 手 脚 并 着 ， 在 湖 泊 中 上 下 窜 来 窜 去 ， 心 想 他 是 一 个 英 雄 好 汉 ， 如 今 可 不 得 让 他 为 难 ， 那 是 不 能 ， 不 然 要 是 真 抢 到 了 他 手 脚 甚 么 。 骆 驼 手 中 的 缰 绳 ， 一 旦 要 去 ， 他 马 上 翻 身 ， 手 脚 一 横 ， 越 想 越 。 他 又 一 回 头 ， 眼 望 大 抬 头 望 去 ， 望 得 是 一 片 旷 野 ， 